In [8]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import transpile
import time

from fast_generator import fc_tree_commute_recur_lookahead_fast

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

In [9]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, threshold = 1):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
            end_time = time.time()
            combined_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results_hardware_topology/test_new_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results_hardware_topology/test_new_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [10]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, threshold = 1, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results_hardware_topology/test_new_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results_hardware_topology/test_new_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [11]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20,]# 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, threshold = 1, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.8522214889526367, 'combined_time': 0.5211782455444336}, 'gate_counts': {'our_method': 99, 'combined_method': 28}, 'circuit_entangling_depth': {'our_method': 69, 'combined_method': 22}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 2.0131237506866455, 'combined_time': 1.3821959495544434}, 'gate_counts': {'our_method': 446, 'combined_method': 191}, 'circuit_entangling_depth': {'our_method': 306, 'combined_method': 164}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 7.859485626220703, 'combined_time': 5.250932931900024}, 'gate_counts': {'our_method': 1962, 'combined_method': 999}, 'circuit_entangling_depth': {'our_method': 1283, 'combined_method': 770}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_Paulis320.json'}
{'num_paulis': 1656, 'times': {'our_time': 50.98212075233

In [12]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, threshold = 1)

benzene.json
{'num_paulis': 1254, 'times': {'our_time': 55.87291216850281, 'combined_time': 52.09362244606018}, 'gate_counts': {'our_method': 17569, 'combined_method': 10090}, 'circuit_entangling_depth': {'our_method': 8842, 'combined_method': 6279}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 7.591831207275391, 'combined_time': 5.271944284439087}, 'gate_counts': {'our_method': 1453, 'combined_method': 789}, 'circuit_entangling_depth': {'our_method': 884, 'combined_method': 630}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_H2O.json'}
LiH.json
{'num_paulis': 61, 'times': {'our_time': 2.392238140106201, 'combined_time': 1.0513362884521484}, 'gate_counts': {'our_method': 244, 'combined_method': 158}, 'circuit_entangling_depth': {'our_method': 188, 'combined_method': 147}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_LiH.json'}


In [13]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 1)

max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'our_time': 48.069353342056274, 'combined_time': 44.474578619003296}, 'gate_counts': {'our_method': 6497, 'combined_method': 3828}, 'circuit_entangling_depth': {'our_method': 3132, 'combined_method': 2606}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_max_cut_benchmark_n10_e45_l20.json'}
max_cut_benchmark_n14_e60_l5.json
{'num_paulis': 370, 'times': {'our_time': 22.271981477737427, 'combined_time': 21.241143941879272}, 'gate_counts': {'our_method': 3886, 'combined_method': 1627}, 'circuit_entangling_depth': {'our_method': 1651, 'combined_method': 926}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_max_cut_benchmark_n14_e60_l5.json'}


In [14]:
#since in the maxcut problems there are many commuting terms and it's slow to search through all of them,
#we can set the threshold to 2 to early stop the search once found a simplified pauli of weight 2
#you can see the run time difference, for the second benchmark runtime reduces from 30second to 7 seconds.

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 2)

max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'our_time': 22.32520842552185, 'combined_time': 19.845942735671997}, 'gate_counts': {'our_method': 5085, 'combined_method': 3406}, 'circuit_entangling_depth': {'our_method': 2406, 'combined_method': 2359}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_max_cut_benchmark_n10_e45_l20.json'}
max_cut_benchmark_n14_e60_l5.json
{'num_paulis': 370, 'times': {'our_time': 15.574185371398926, 'combined_time': 16.234165906906128}, 'gate_counts': {'our_method': 3914, 'combined_method': 1618}, 'circuit_entangling_depth': {'our_method': 1615, 'combined_method': 926}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_max_cut_benchmark_n14_e60_l5.json'}


In [15]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, threshold= 2)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 3.417402505874634, 'combined_time': 3.739920139312744}, 'gate_counts': {'our_method': 802, 'combined_method': 389}, 'circuit_entangling_depth': {'our_method': 439, 'combined_method': 268}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_labs_n10_layers1.json'}
labs_n10_layers3.json
{'num_paulis': 240, 'times': {'our_time': 11.759809970855713, 'combined_time': 7.47677206993103}, 'gate_counts': {'our_method': 2216, 'combined_method': 1128}, 'circuit_entangling_depth': {'our_method': 1149, 'combined_method': 788}, 'test_paulis_file': 'benchmarks/results_hardware_topology/test_new_labs_n10_layers3.json'}
labs_n11_layers1.json
{'num_paulis': 106, 'times': {'our_time': 13.81656527519226, 'combined_time': 5.882989406585693}, 'gate_counts': {'our_method': 1226, 'combined_method': 534}, 'circuit_entangling_depth': {'our_method': 573, 'combined_method': 373}, 'test_paulis_file': 'benchmarks/results_hardware_topolog